In [2]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

In [3]:
# Importing data
df = pd.read_csv("extracted_by_year/2006_extracted/2006tmax.csv")
df.head()

# set the cutoff
CUTOFF = 32.2

,OID_,ZCTA5CE20,AFFGEOID20,GEOID20,NAME20,LSAD20,ALAND20,AWATER20,ORIG_FID,tmaxJan01,...,tmaxDec22,tmaxDec23,tmaxDec24,tmaxDec25,tmaxDec26,tmaxDec27,tmaxDec28,tmaxDec29,tmaxDec30,tmaxDec31
0,1,15301,860Z200US15301,15301,15301,Z5,315861121.0,709775.0,0,5.577000,...,8.703,11.922,7.904000,6.572,8.467,7.580000,4.295,7.025,11.838,12.255000
1,2,15658,860Z200US15658,15658,15658,Z5,238683518.0,759690.0,1,4.707000,...,8.455,11.095,8.572000,7.214,8.788,7.958000,4.449,7.661,11.458,12.299000
2,3,15601,860Z200US15601,15601,15601,Z5,208874774.0,337008.0,2,4.968000,...,8.779,11.747,8.216000,6.497,8.777,8.112001,3.893,7.177,11.703,12.038000
3,4,30114,860Z200US30114,30114,30114,Z5,246536321.0,17044903.0,5,14.021000,...,12.283,16.941,16.240999,16.254,10.649,4.665000,10.636,13.064,15.203,14.262999
4,5,30281,860Z200US30281,30281,30281,Z5,179405961.0,2749692.0,6,16.273001,...,13.935,17.869,17.568001,15.255,14.067,6.109000,11.534,13.873,16.972,13.772000


In [4]:
# reshape, remove unnecessary columns and add statistical filter + threshold
def proc(frame, upper_threshold):
    processed = frame.copy()
    for name in list(frame):
        if name[:4] != "tmax" and name != "GEOID20":
            processed = processed.drop(name, axis=1)
    
        
    stats = processed.set_index('GEOID20')
    stats = stats.apply(pd.DataFrame.describe, axis=1)
    stats["upper"] = stats["mean"]+1.645*stats["std"]

    processed = processed.melt(id_vars='GEOID20', var_name='Date', value_name='Temp')
    
    # conversion to Python datetime objects
    processed['Date'] = processed['Date'].map(lambda x: x.lstrip('tmax')) + "2006"
    processed['Date'] = pd.to_datetime(processed['Date'], format='%b%d%Y')

    # Set + sort by multi-level index using 'GEOID20' and 'Date'
    processed.set_index(['GEOID20', 'Date'], inplace=True)
    processed.sort_index(inplace=True)

    thresholds = stats.copy()
    thresholds.drop(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], inplace=True)
    
    # for upper here we currently have this as 32.2
    thresholds['static']= upper_threshold
    
    merged = processed.reset_index().merge(thresholds.reset_index(), on='GEOID20', how='left').set_index(['GEOID20', 'Date'])

    return merged

In [5]:
# function to create a (not indexed) boolean series which tells you whether an entry is part of a consecutive series of length n
def consec_count_n(series, n):
    temp = []
    c = 0
    for i in range(len(series.values)):
        if series.values[i] == False:
            c = 0
        else:
            c += 1
        temp.append(c)
    
    result = []
    pass_thres = False
    for i in range(len(temp)):
        if temp[-(i+1)] >= n:
            pass_thres = True
            result.append(True)
        elif temp[-(i+1)] != 0 and pass_thres:
            result.append(True)
        else:
            pass_thres = False
            result.append(False)
    result = result[::-1]

    return pd.Series(result)

In [6]:
# function to check if the contig_mask (consec_count function) is working
def check(frame, contig_mask):
    threshold_mask = (frame['Temp'] > frame['upper']) | (frame['Temp'] > frame['static'])

    result = pd.DataFrame(contig_mask)

    result.reset_index()
    result.columns = ["contiguous"]
    result.index = frame.index

    check = pd.DataFrame(result)
    check.insert(1, "true-false", threshold_mask)
    
    with pd.option_context('display.max_rows', None,):
        print(check)

In [7]:
# function which returns a filtered version of a given dataframe which only includes the entries which are part of a consecutive series of length n Trues for a boolean mask
def consec_filter(frame, n):
    threshold_mask = (frame['Temp'] > frame['upper']) | (frame['Temp'] > frame['static'])
    
    contig_mask = pd.DataFrame(data=consec_count_n(threshold_mask, n))

    contig_mask.reset_index()
    contig_mask.columns = ["contiguous"]
    contig_mask.index = frame.index

    filtered_data = frame[frame.index.isin(contig_mask.index[contig_mask['contiguous']])]

    return filtered_data

In [29]:
# function to turn the dataframe of days into flat data
def flatten(frame):
    sequences = []

    # loop to find and append the sequences and their lengths to a new array
    for zip_code, group in frame.groupby(level='GEOID20'):
        start_date = None
        sequence_length = 0
        prev_date = None
        for date in group.index.get_level_values('Date'):
            if start_date is None:
                start_date = date
                sequence_length = 1
            elif prev_date is not None and (date - prev_date).days == 1:
                sequence_length += 1
            else:
                sequences.append({
                    'GEOID20': zip_code,
                    'Start_Date': start_date.date(),
                    'Sequence_Length': sequence_length
                })
                start_date = date
                sequence_length = 1
            prev_date = date
        if sequence_length > 0:
            sequences.append({
                'GEOID20': zip_code,
                'Start_Date': start_date.date(),
                'Sequence_Length': sequence_length
            })
    # flatten to 2d array
    result = [[seq['GEOID20'], seq['Start_Date'], seq['Sequence_Length']] for seq in sequences]

    return result

In [ ]:
# function to do the whole process
def find_heatwaves(frame, cutoff):
    processed = proc(frame, cutoff)
    

In [28]:
processed_2006 = proc(df, 32.2)
print(processed_2006)

                     Temp      upper  static
GEOID20 Date                                
1001    2006-01-01 -2.041  32.610467    32.2
        2006-01-02 -0.211  32.610467    32.2
        2006-01-03  3.070  32.610467    32.2
        2006-01-04  1.140  32.610467    32.2
        2006-01-05 -0.879  32.610467    32.2
...                   ...        ...     ...
99403   2006-12-27  5.746  34.555681    32.2
        2006-12-28  5.304  34.555681    32.2
        2006-12-29  2.147  34.555681    32.2
        2006-12-30  2.066  34.555681    32.2
        2006-12-31  2.168  34.555681    32.2

[12154500 rows x 3 columns]


In [30]:
test_2 = consec_filter(processed_2006.loc[ 1001: 1002], 3)
print(test_2)

                         Temp      upper  static
GEOID20 Date                                    
1001    2006-07-17  34.178001  32.610467    32.2
        2006-07-18  35.014000  32.610467    32.2
        2006-07-19  35.320999  32.610467    32.2
        2006-08-02  35.296001  32.610467    32.2
        2006-08-03  37.244000  32.610467    32.2
        2006-08-04  34.931000  32.610467    32.2
1002    2006-07-17  33.160999  31.517748    32.2
        2006-07-18  33.617001  31.517748    32.2
        2006-07-19  33.664001  31.517748    32.2
        2006-08-02  34.471001  31.517748    32.2
        2006-08-03  35.634998  31.517748    32.2
        2006-08-04  32.222000  31.517748    32.2


In [31]:
# Extract lengths of consecutive sequences
testresult = flatten(test_2)

print(testresult)

[[1001, datetime.date(2006, 7, 17), 3], [1001, datetime.date(2006, 8, 2), 3], [1002, datetime.date(2006, 7, 17), 3], [1002, datetime.date(2006, 8, 2), 3]]


In [23]:
# export file
np.savetxt("heatwave_instances_test.csv", testresult, fmt='%s', delimiter=", ", header=', '.join(['Zip_Code', 'Starting_Date', 'Days']))

In [32]:
days_2006 = consec_filter(processed_2006, 3)
result_2006 = flatten(days_2006)

#with pd.option_context('display.max_rows', None,):
#    print(days_of_interest)

In [27]:
# format the headers of the columns
columnNames = ['Zip_Code', 'Starting_Date', 'Days']

# export 2006
np.savetxt("heatwave_2006.csv", result_2006, fmt='%s', delimiter=', ', header=', '.join(columnNames))